Verification of 3.2.1 Stochiometric Surfaces

start by creating surfaces with flourite structure

In [2]:
from ase.io import read, write
from ase.build import surface, make_supercell
from pymatgen.core.surface import SlabGenerator
from pymatgen.core.structure import Structure

structure = read("bulk.cif")

In [22]:
# (111) surface
slab = surface(structure, (1,1,1), layers=14, vacuum=0.0)
supercell = make_supercell(slab,[[2,0,0],[0,2,0],[0,0,1]])
#write('111slab.cif', slab, format='cif')
write('111supercell.cif', supercell, format='cif')

def get_cell_length_c(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('_cell_length_c'):
                # Extract the value after the key
                _, value = line.split()
                return float(value)
    raise ValueError("_cell_length_c not found in the file.")

file_path = '111supercell.cif'
min_slab_size = get_cell_length_c(file_path)

bulk_structure = Structure.from_file("bulk.cif")

# Generate slabs with different termination types
slabgen = SlabGenerator(
    bulk_structure,
    miller_index=(1, 1, 1),  # Miller index of the surface plane
    min_slab_size=min_slab_size,      # Minimum slab thickness in Angstroms
    min_vacuum_size=120.0,    # Minimum vacuum size in Angstroms
    lll_reduce=False         # Whether to use the LLL algorithm to reduce the cell
)

slabs = slabgen.get_slabs(
    symmetrize=True,     # Whether to symmetrize the slab
    tol=0.1,              # Symmetrization tolerance
    bonds=None,           # Custom bonds to pass to the slab generator
    max_broken_bonds=0   # Maximum number of broken bonds allowed in the slab
)

# Create a 4x4 supercell in the xy-axis for each slab and save them
for i, slab in enumerate(slabs):
    # Create the 2x2 supercell for the slab
    supercell_scaling_matrix = [[4, 0, 0], [0, 4, 0], [0, 0, 1]]
    supercell = slab.make_supercell(supercell_scaling_matrix)
    # Check if the total number of atoms in the supercell is equal to 336
    if len(supercell) == 672:
        # Optionally, write the supercell slab to a file named 111slab.cif only if the condition is met
        supercell.to(fmt="cif", filename="111slab.cif")
        print(f"Supercell Slab {i+1} with 672 atoms saved as 111slab.cif")
    else:
        print(f"Supercell Slab {i+1} does not have 672 atoms and was not saved.")


Supercell Slab 1 does not have 672 atoms and was not saved.
Supercell Slab 2 with 672 atoms saved as 111slab.cif
Supercell Slab 3 does not have 672 atoms and was not saved.


In [4]:
# (110) surface
slab = surface(structure, (1,1,0), layers=20, vacuum=0.0)
supercell = make_supercell(slab,[[2,0,0],[0,2,0],[0,0,1]])
#write('111slab.cif', slab, format='cif')
write('110supercell.cif', supercell, format='cif')

def get_cell_length_c(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('_cell_length_c'):
                # Extract the value after the key
                _, value = line.split()
                return float(value)
    raise ValueError("_cell_length_c not found in the file.")

file_path = '110supercell.cif'
min_slab_size = get_cell_length_c(file_path)

bulk_structure = Structure.from_file("bulk.cif")

# Generate slabs with different termination types
slabgen = SlabGenerator(
    bulk_structure,
    miller_index=(1, 1, 0),  # Miller index of the surface plane
    min_slab_size=min_slab_size,      # Minimum slab thickness in Angstroms
    min_vacuum_size=120.0,    # Minimum vacuum size in Angstroms
    lll_reduce=False         # Whether to use the LLL algorithm to reduce the cell
)

slabs = slabgen.get_slabs(
    symmetrize=True,     # Whether to symmetrize the slab
    tol=0.1,              # Symmetrization tolerance
    bonds=None,           # Custom bonds to pass to the slab generator
    max_broken_bonds=0   # Maximum number of broken bonds allowed in the slab
)

# Create a 4x4 supercell in the xy-axis for each slab and save them
for i, slab in enumerate(slabs):
    # Create the 2x2 supercell for the slab
    supercell_scaling_matrix = [[4, 0, 0], [0, 4, 0], [0, 0, 1]]
    supercell = slab.make_supercell(supercell_scaling_matrix)
    # Check if the total number of atoms in the supercell is equal to 336
    supercell.to(fmt="cif", filename="110slab.cif")
    

In [2]:
#new110slab
from ase.io import read, write
from ase.build import surface, make_supercell

slab = read("geometry_file/110slab.cif")
supercell = make_supercell(slab,[[2,0,0],[0,1,0],[0,0,1]])
bottom_z = 75.0
front_x = 3.86
supercell = supercell[[atom.index for atom in supercell if atom.position[2] > bottom_z]]
supercell = supercell[[atom.index for atom in supercell if atom.position[0] > front_x]]
write('new110slab.cif',supercell,format='cif')


In [ ]:
# (100) surface
slab = surface(structure, (1,0,0), layers=9, vacuum=10.0)
slab.center(vacuum=15.0, axis=2)

for i in range(2):
    top_layer_z = max(slab.positions[:, 2])
    slab = slab[[atom.index for atom in slab if atom.position[2] < top_layer_z]]

supercell = make_supercell(slab,[[4,0,0],[0,4,0],[0,0,1]])
#write('geometry_file/100slab.cif', slab, format='cif')
#write('geometry_file/100supercell.cif', supercell, format='cif')

In [20]:
def add_charges_to_atoms(input_file, output_file):
    # Define the charges for each atom type
    atom_charges = {
        1: 4.0,  # Ce
        2: -2.0  # O
    }

    # Read the file and store its contents
    with open(input_file, "r") as file:
        lines = file.readlines()

    # Find the line number where atom information starts
    atom_info_line = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Atoms"):
            atom_info_line = i + 1
            break

    # If the line number is found, add charges to each atom
    if atom_info_line is not None:
        # Iterate through atom lines and add charges
        for i in range(atom_info_line, len(lines)):
            atom_line = lines[i].split()
            if len(atom_line) < 2:
                continue  # Skip lines that do not have enough elements
            atom_index = atom_line[0]
            atom_type = int(atom_line[1])
            charge = atom_charges.get(atom_type)
            if charge is not None:
                # Modify the line to include charge
                lines[i] = f"         {atom_line[0]}         {atom_line[1]}   {charge}        {atom_line[2]}       {atom_line[3]}       {atom_line[4]}\n"

    # Write the modified contents back to the output file
    with open(output_file, "w") as file:
        file.writelines(lines)

Manually run this function only once to add charges for slab.lmp

In [23]:
#add_charges_to_atoms("geometry_file/111supercell.lmp","geometry_file/111supercell.lmp")
#add_charges_to_atoms("geometry_file/110supercell.lmp","geometry_file/110supercell.lmp")
#add_charges_to_atoms("geometry_file/100slab.lmp","geometry_file/100slab.lmp")
#add_charges_to_atoms("geometry_file/111slab.lmp","geometry_file/e111slab.lmp")
#add_charges_to_atoms("geometry_file/new110slab.lmp","geometry_file/new110slab.lmp")
#add_charges_to_atoms("new110supercell.lmp","new110supercell.lmp")
#add_charges_to_atoms("111slab.lmp","111slab.lmp")